In [8]:
import cv2

# 读取图片
image_np = cv2.imread("../image/road.png")

# 灰度图
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# 二值化可写可不写
res,image_thresh = cv2.threshold(image_gray,127,255,cv2.THRESH_BINARY)

# 高斯滤波
# image_gauss = cv2.GaussianBlur(image_gray, (5, 5), 1.5)
image_gauss = cv2.GaussianBlur(image_thresh, (5, 5), 1.5)

# 进行边缘检测
image_canny = cv2.Canny(image_gauss, 30, 70)

# 结果显示
cv2.imshow("image_canny", image_canny)
cv2.waitKey(0)

-1

In [ ]:
# -*- coding: utf-8 -*-
import cv2

# ==============================
# 1) 读取图片
# ==============================
# cv2.imread(filename, flags) -> ndarray
#  - filename: 图像路径
#  - flags: 读取方式
#       cv2.IMREAD_COLOR(默认)  : 以 BGR 彩色读入（忽略 alpha）
#       cv2.IMREAD_GRAYSCALE    : 以单通道灰度读入
#       cv2.IMREAD_UNCHANGED    : 保留 alpha 通道
# 返回值为 numpy 数组，shape = (H, W, 3) 或 (H, W)
image_np = cv2.imread("../image/road.png")
if image_np is None:
    raise FileNotFoundError("未找到图片，请检查路径 ../image/road.png")

# ==============================
# 2) 转灰度
# ==============================
# cv2.cvtColor(src, code) -> ndarray
#  - src : 输入图像
#  - code: 颜色空间转换码，如：
#       cv2.COLOR_BGR2GRAY  BGR -> Gray
#       cv2.COLOR_BGR2RGB   BGR -> RGB
# 说明：Canny 在灰度图上工作；灰度能降低计算量并避免通道干扰。
image_gray = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)

# ==============================
# 3) 二值化（可选）
# ==============================
# cv2.threshold(src, thresh, maxval, type) -> (ret, dst)
#  - src    : 输入单通道图（通常灰度）
#  - thresh : 阈值（分界点）
#  - maxval : 赋给“超过/低于阈值”的像素值（与 type 相关，常取 255）
#  - type   : 阈值类型，如：
#       cv2.THRESH_BINARY         : 大于阈值设为 maxval，否则为 0
#       cv2.THRESH_BINARY_INV     : 反二值
#       cv2.THRESH_OTSU           : Otsu 自动阈值（与 THRESH_BINARY 按位或）
# 返回：
#  - ret : 实际使用的阈值（Otsu 时为自动求得的阈值）
#  - dst : 阈值化后的图像
# 备注：Canny 前“必须”高斯平滑，但是否二值化并非必须；二值化会让边缘更硬，
#      对路面场景未必更好，一般建议直接对灰度做平滑 + Canny。
res, image_thresh = cv2.threshold(image_gray, 127, 255, cv2.THRESH_BINARY)

# ==============================
# 4) 高斯滤波（降噪）
# ==============================
# cv2.GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
#  - src     : 输入图像（可以是灰度或二值，通常直接用灰度）
#  - ksize   : 卷积核大小 (宽, 高)，必须为奇数，如 (3,3) / (5,5) / (7,7)
#  - sigmaX  : X 方向高斯标准差（控制平滑强度）；为 0 时由 ksize 估计
#  - sigmaY  : Y 方向标准差（默认 0，表示与 sigmaX 相同）
#  - borderType: 边界填充方式，默认 cv2.BORDER_DEFAULT
# 作用：抑制随机噪声，避免噪声在 Canny 中被当作边缘。
# 调参：核越大/σ越大，越平滑但边缘越钝。路面建议 (5,5) + sigma≈1.0~1.5
# 说明：如果你希望保持更多细节，可改用 image_gray 作为输入而不是 image_thresh。
# image_gauss = cv2.GaussianBlur(image_gray, (5, 5), 1.5)
image_gauss = cv2.GaussianBlur(image_thresh, (5, 5), 1.5)

# ==============================
# 5) Canny 边缘检测
# ==============================
# cv2.Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) -> edges
#  - image        : 输入单通道图（推荐是平滑后的灰度图/二值图）
#  - threshold1   : 低阈值（弱边缘下限）
#  - threshold2   : 高阈值（强边缘下限，通常是低阈值的 2~3 倍）
#  - apertureSize : Sobel 核大小（3/5/7，默认 3）；越大越平滑、边变钝
#  - L2gradient   : True 用精确范数 sqrt(gx^2+gy^2)，False 用近似 |gx|+|gy|
# 返回：二值边缘图（0=非边缘，255=边缘）
# 调参经验：
#   - 边缘断裂多 → 降低 threshold2 或减小平滑强度
#   - 噪点多     → 提高 threshold2 或增大平滑强度/核
#   - 细节不够   → 使用 L2gradient=True，或减小高斯核
image_canny = cv2.Canny(image_gauss, 30, 70)
# 等价更完整写法示例：
# image_canny = cv2.Canny(image_gauss, 30, 70, apertureSize=3, L2gradient=False)

# ==============================
# 6) 结果展示
# ==============================
# cv2.imshow(winname, mat)    : 显示窗口
# cv2.waitKey(delay)          : 键盘等待，单位 ms；0 表示无限等待
# cv2.destroyAllWindows()     : 关闭所有窗口
cv2.imshow("image_canny", image_canny)
cv2.waitKey(0)
cv2.destroyAllWindows()
